In [1]:
from utils import *
from sklearn.cross_validation import train_test_split
import time

/home/husein/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [7006, 300, 6, 62, 2165, 3661, 1641, 1122, 99, 5347] ['oddity', 'sure', 'one', 'might', 'wind', 'remembering', 'degree', 'affection', 'rather', 'revulsion']


In [4]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [5]:
class Model:
    def __init__(self, size_layer, embed_size, dict_size, dimension_output, learning_rate = 1e-3):
        self.X = tf.placeholder(tf.int32, [None, 100])
        self.Y = tf.placeholder(tf.int32, [None])
        
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embed_size], -1, 1))
        x = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        def cells(reuse=False):
            return tf.contrib.rnn.DropoutWrapper(
            tf.nn.rnn_cell.GRUCell(size_layer, reuse=reuse), output_keep_prob = 0.8,
                input_keep_prob = 0.8
        )
            
        batch_size = tf.shape(self.X)[0]
        
        x = tf.reshape(x, (batch_size*10, 10, size_layer))
        with tf.variable_scope('1'):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = x)
            rnn_cell = tf.contrib.seq2seq.AttentionWrapper(cell = cells(),
                                                           attention_mechanism = attention_mechanism,
                                                           attention_layer_size = size_layer)
            x, _ = tf.nn.dynamic_rnn(
                cell = rnn_cell, 
                inputs = x,
                dtype = tf.float32)
        x = tf.reduce_max(x, 1)
        
        x = tf.reshape(x, (batch_size, 10, size_layer))
        with tf.variable_scope('2'):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = x)
            rnn_cell = tf.contrib.seq2seq.AttentionWrapper(cell = cells(),
                                                          attention_mechanism = attention_mechanism,
                                                           attention_layer_size = size_layer)
            x, _ = tf.nn.dynamic_rnn(
                cell = rnn_cell, 
                inputs = x,
                dtype = tf.float32)
        x = tf.reduce_max(x, 1)
        
        x = tf.reshape(x, (batch_size, 1, size_layer))
        with tf.variable_scope('3'):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = x)
            rnn_cell = tf.contrib.seq2seq.AttentionWrapper(cell = cells(),
                                                          attention_mechanism = attention_mechanism,
                                                           attention_layer_size = size_layer)
            x, _ = tf.nn.dynamic_rnn(
                cell = rnn_cell, 
                inputs = x,
                dtype = tf.float32)
        x = tf.reduce_max(x, 1)
        self.logits = tf.layers.dense(x, dimension_output)

        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [6]:
size_layer = 200
embed_size = 200
maxlen = 100
batch_size = 32
dimension_output = len(trainset.target_names)

In [7]:
import tensorflow as tf

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer, embed_size, len(dictionary), dimension_output)
sess.run(tf.global_variables_initializer())

In [9]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [10]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   1%|          | 2/267 [00:00<00:18, 14.16it/s, accuracy=0.5, cost=0.644]

epoch: 0, pass acc: 0.000000, current acc: 0.640346
time taken: 20.976581573486328
epoch: 0, training loss: 0.713305, training acc: 0.564158, valid loss: 0.650553, valid acc: 0.640346



train minibatch loop:   1%|          | 2/267 [00:00<00:18, 14.43it/s, accuracy=0.812, cost=0.429]

epoch: 1, pass acc: 0.640346, current acc: 0.690286
time taken: 20.294590711593628
epoch: 1, training loss: 0.547850, training acc: 0.726870, valid loss: 0.645137, valid acc: 0.690286



train minibatch loop:   1%|          | 2/267 [00:00<00:18, 14.63it/s, accuracy=0.875, cost=0.347]

...

time taken: 20.186052560806274
epoch: 4, training loss: 0.165602, training acc: 0.938342, valid loss: 1.381705, valid acc: 0.690487



train minibatch loop:   1%|          | 2/267 [00:00<00:18, 14.48it/s, accuracy=0.938, cost=0.165]

time taken: 20.220810174942017
epoch: 5, training loss: 0.113889, training acc: 0.960371, valid loss: 1.241685, valid acc: 0.681825



test minibatch loop: 100%|██████████| 67/67 [00:01<00:00, 35.83it/s, accuracy=0.667, cost=0.434]

time taken: 20.230729818344116
epoch: 6, training loss: 0.084232, training acc: 0.971771, valid loss: 1.173060, valid acc: 0.680888

break epoch:7

